In [ ]:
feq = listings[listings['accommodates']==8]
feq = listings[listings['neighbourhood_group'] == 'Kauai']
feq = feq.groupby('neighbourhood')['price'].mean().sort_values(ascending=True)
feq.plot.barh(figsize=(10, 8), color=['#454545', '#6d5959', '#9dcbba', '#f3de8a', '#eb9486', '#7e7f9a', '#8377d1', '#a53860'],edgecolor = 'black', width=1)
plt.title("Precio medio diario para 8 personas en Kauai", fontsize=20)
plt.xlabel('Precio medio ($)', fontsize=12)
plt.ylabel("")
plt.show();

feq = listings[listings['accommodates']==2] 
feq = feq.groupby('neighbourhood')['price'].mean().sort_values(ascending=True) 
adam = gpd.read_file("/Users/pedrollamaslopez/Desktop/Curso_upgrade/Módulo_2/16-Data StoryTelling/Hawaii/data/neighbourhoods.geojson") 
feq = pd.DataFrame([feq]) 
feq = feq.transpose() 

adam = pd.merge(adam, feq, on='neighbourhood', how='left') 
adam.rename(columns={'price': 'average_price'}, inplace=True) 
adam.average_price = adam.average_price.round(decimals=0) 
adam = adam.dropna(subset=['average_price']) 

map_dict = adam.set_index('neighbourhood')['average_price'].to_dict() 
color_scale = LinearColormap(['yellow','red','purple'], vmin=min(map_dict.values()), vmax=max(map_dict.values()), caption='Average price') 
def get_color(feature): 
    value = map_dict.get(feature['properties']['neighbourhood']) 
    if value is None: 
        return '#BADADA' 
    else: 
        return color_scale(value) 
    
map3 = folium.Map(location=[22.073414994647898, -159.5257299787407], zoom_start=10) 
folium.GeoJson(data=adam, 
               name='Hawaii', 
               tooltip=folium.features.GeoJsonTooltip(fields=['neighbourhood', 'average_price'],
                                                      labels=True, 
                                                      sticky=False), 
               style_function= lambda feature: {
                   'fillColor': get_color(feature),
                   'color': 'black',
                   'weight': 1, 
                   'dashArray': '5, 5', 
                   'fillOpacity':0.9 }, 
               highlight_function=lambda feature: {'weight':3, 'fillColor': get_color(feature), 'fillOpacity': 0.8}).add_to(map3) 
map3.add_child(color_scale)
Contraer
tiene menú contextual



In [ ]:
import nltk
nltk.download('stopwords')
#take out empty comments (530)
reviews_details = reviews_details[reviews_details['comments'].notnull()]

#remove numbers
reviews_details['comments'] = reviews_details['comments'].str.replace('\d+', '') 
#all to lowercase
reviews_details['comments'] = reviews_details['comments'].str.lower()
#remove windows new line
reviews_details['comments'] = reviews_details['comments'].str.replace('\r\n', "")
#remove stopwords (from nltk library)
stop_english = stopwords.words("english")
reviews_details['comments'] = reviews_details['comments'].apply(lambda x: " ".join([i for i in x.split() 
                                                      if i not in (stop_english)]))
# remove punctuation
reviews_details['comments'] = reviews_details['comments'].str.replace('[^\w\s]'," ")
# replace x spaces by one space
reviews_details['comments'] = reviews_details['comments'].str.replace('\s+', ' ')

reviews_details.comments.values[2] #print same comments again

texts = reviews_details.comments.tolist()

vec = CountVectorizer().fit(texts)
bag_of_words = vec.transform(texts)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

cvec_df = pd.DataFrame.from_records(words_freq, columns= ['words', 'counts']).sort_values(by="counts", ascending=False)
cvec_df.head(10)

cvec_dict = dict(zip(cvec_df.words, cvec_df.counts))

wordcloud = WordCloud(width=800, height=400)
wordcloud.generate_from_frequencies(frequencies=cvec_dict)
plt.figure( figsize=(20,10) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
Contraer
tiene menú contextual